In [ ]:
import re
import markdown
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.auth import default

# Authenticate and initialize the Google Docs API
def authenticate_google_docs():
    auth.authenticate_user()
    creds, _ = default()
    return build("docs", "v1", credentials=creds)

# Create a new Google Doc programmatically
def create_new_google_doc(service, title):
    try:
        doc = service.documents().create(body={"title": title}).execute()
        doc_id = doc["documentId"]
        print(f"Document created successfully: https://docs.google.com/document/d/{doc_id}")
        return doc_id
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

# Parse markdown and create structured Google Docs content
def parse_markdown_to_gdocs(markdown_text):
    lines = markdown_text.split("\n")
    requests = []
    current_location = 1  # Start from the beginning of the document

    for line in lines:
        # Main title (Heading 1)
        if line.startswith("# "):
            text = line[2:] + "\n\n"
            requests.append({
                "insertText": {"text": text, "location": {"index": current_location}}
            })
            start_index = current_location
            current_location += len(text)
            requests.append({
                "updateParagraphStyle": {
                    "paragraphStyle": {"namedStyleType": "HEADING_1"},
                    "range": {"startIndex": start_index, "endIndex": current_location},
                    "fields": "*"
                }
            })

        # Section headers (Heading 2)
        elif line.startswith("## "):
            text = line[3:] + "\n\n"
            requests.append({
                "insertText": {"text": text, "location": {"index": current_location}}
            })
            start_index = current_location
            current_location += len(text)
            requests.append({
                "updateParagraphStyle": {
                    "paragraphStyle": {"namedStyleType": "HEADING_2"},
                    "range": {"startIndex": start_index, "endIndex": current_location},
                    "fields": "*"
                }
            })

        # Sub-section headers (Heading 3)
        elif line.startswith("### "):
            text = line[4:] + "\n\n"
            requests.append({
                "insertText": {"text": text, "location": {"index": current_location}}
            })
            start_index = current_location
            current_location += len(text)
            requests.append({
                "updateParagraphStyle": {
                    "paragraphStyle": {"namedStyleType": "HEADING_3"},
                    "range": {"startIndex": start_index, "endIndex": current_location},
                    "fields": "*"
                }
            })

        # Convert markdown checkboxes "- [ ]" to Google Docs checkboxes
        elif line.startswith("- [ ]"):
            text = "\u2610 " + line[6:] + "\n"
            requests.append({
                "insertText": {"text": text, "location": {"index": current_location}}
            })
            current_location += len(text)

        # Bullet points
        elif re.match(r"^- ", line):
            text = "• " + line[2:] + "\n"
            requests.append({
                "insertText": {"text": text, "location": {"index": current_location}}
            })
            current_location += len(text)

        # Assignee mentions (e.g., @sarah, @mike, etc.)
        elif re.search(r"@[\w]+", line):
            assignees = re.findall(r"@[\w]+", line)  # Find all mentions in the line
            text = line + "\n"
            requests.append({
                "insertText": {"text": text, "location": {"index": current_location}}
            })
            start_index = current_location
            current_location += len(text)

            # For each assignee mention, apply bold
            for assignee in assignees:
                assignee_start_index = text.find(assignee, start_index)
                assignee_end_index = assignee_start_index + len(assignee)

                # Apply bold
                requests.append({
                    "updateTextStyle": {
                        "textStyle": {
                            "bold": True,
                        },
                        "range": {"startIndex": assignee_start_index, "endIndex": assignee_end_index},
                        "fields": "bold,foregroundColor"
                    }
                })

        # Footer information (Meeting recorded by, Duration) - Italic style and center alignment
        elif line.startswith("Meeting recorded by") or line.startswith("Duration"):
            text = line + "\n"
            requests.append({
                "insertText": {"text": text, "location": {"index": current_location}}
            })
            start_index = current_location
            current_location += len(text)

            # Apply italic style to footer text
            requests.append({
                "updateTextStyle": {
                    "textStyle": {"italic": True},
                    "range": {"startIndex": start_index, "endIndex": current_location},
                    "fields": "italic"  # Specify only the 'italic' field
                }
            })

            # Apply center alignment to footer paragraph
            requests.append({
                "updateParagraphStyle": {
                    "paragraphStyle": {
                        "alignment": "CENTER",  # Center-aligned text
                    },
                    "range": {"startIndex": start_index, "endIndex": current_location},
                    "fields": "alignment"  # Specify only the 'alignment' field
                }
            })


        # Regular text
        else:
            text = line + "\n"
            requests.append({
                "insertText": {"text": text, "location": {"index": current_location}}
            })
            current_location += len(text)

    return requests




# Create a new Google Doc
def create_google_doc(service, title, content_requests):
    doc_id = create_new_google_doc(service, title)  # Create the doc
    if doc_id:
        try:
            # Apply all the requests to the created document
            service.documents().batchUpdate(documentId=doc_id, body={"requests": content_requests}).execute()
            print(f"Content added successfully to document: https://docs.google.com/document/d/{doc_id}")
        except HttpError as error:
            print(f"An error occurred: {error}")

# Markdown notes
markdown_notes = """# Product Team Sync - May 15, 2023

## Attendees

- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items

- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---

Meeting recorded by: Sarah Chen

Duration: 45 minutes
"""

# Run the process
service = authenticate_google_docs()
content_requests = parse_markdown_to_gdocs(markdown_notes)
create_google_doc(service, "Product Team Sync", content_requests)


Document created successfully: https://docs.google.com/document/d/1oPmaHYhRhP5XPvlakl9-koR9wY9CqewqaOEyrE5ajtk
Content added successfully to document: https://docs.google.com/document/d/1oPmaHYhRhP5XPvlakl9-koR9wY9CqewqaOEyrE5ajtk
